In [1]:
#Put all file in ANomaly of Temperature
#Exclude Outliers
#Save with the right format

import sys,os
import tarfile
import warnings

import numpy as np
import pandas as pd
import xarray as xr

import SDFC.link as sdl
import NSSEA as ns
import NSSEA.plot as nsp
import NSSEA.models as nsm
import scipy.stats as sc
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot_2samples

import cftime
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as mpdf
import matplotlib.patches as mplpatch

In [2]:
basepath=os.path.abspath(os.getcwd())
pathInp='/home/barbauxo/Documents/Doctorat/03_Travail/2023_01 Application Tricastin/Data'

pathOut='/home/barbauxo/Documents/Temporary/Outputs (Test Full_V1 Hadcrut)'
assert(os.path.exists(pathInp))
assert(os.path.exists(pathOut))
time_period    = np.arange( 1850 , 2101 , 1 , dtype = np.int )
time_reference = np.arange( 1986 , 2016 , 1 , dtype = np.int )
type_data_entry="02_Selected"
type_data_output="03_Post_treatment"

/tmp/ipykernel_34665/3626403167.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  time_period    = np.arange( 1850 , 2101 , 1 , dtype = np.int )
/tmp/ipykernel_34665/3626403167.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional informatio

In [40]:
%run ../Scripts/data_preparation.py

In [41]:
lX,lY,models=load_models_CMIP6(pathInp,type_data_entry)
Xo,Yo=load_obs(pathInp,type_data_entry)
#From Absolute to ANomaly
Xo,Yo,lX,lY,bias=Everyone_as_anomaly(Xo,Yo,lX,lY,time_reference)

['ACCESS-CM2_i1p1f1', 'ACCESS-ESM1-5_i1p1f1', 'CMCC-ESM2_i1p1f1', 'CNRM-CM6-1-HR_i1p1f2', 'CNRM-CM6-1_i1p1f2', 'CNRM-ESM2-1_i1p1f2', 'CanESM5_i1p2f1', 'EC-Earth3-CC_i1p1f1', 'EC-Earth3-Veg-LR_i1p1f1', 'EC-Earth3-Veg_i1p1f1', 'EC-Earth3_i1p1f1', 'FGOALS-g3_i1p1f1', 'GFDL-ESM4_i1p1f1', 'HadGEM3-GC31-LL_i1p1f3', 'HadGEM3-GC31-MM_i1p1f3', 'INM-CM4-8_i1p1f1', 'INM-CM5-0_i1p1f1', 'IPSL-CM6A-LR_i1p1f1', 'KACE-1-0-G_i1p1f1', 'MIROC-ES2L_i1p1f2', 'MIROC6_i1p1f1', 'MPI-ESM1-2-LR_i1p1f1', 'MRI-ESM2-0_i1p1f1', 'MRI-ESM2-0_i2p1f1', 'NESM3_i1p1f1', 'NorESM2-MM_i1p1f1', 'TaiESM1_i1p1f1', 'UKESM1-0-LL_i1p1f2']


In [52]:
bias_pd=pd.Series(bias)
bias_pd.to_xarray().to_netcdf( os.path.join( pathOut ,"Bias.nc") )
#.to_netcdf( os.path.join( pathOut ,"Bias.nc") )

In [94]:
#Enlever Outliers pour Y
pathOutY= os.path.join(pathInp,"CMIP6/Y")
log_outliers=open(pathOutY+'/Y_Outliers_log.txt','a')
for i in range(0,len(models)):
    m=models[i]
    Y=lY[i]
    Outliers=is_outlier(Y, m)
    Y_not_anomalies=Outliers[(Outliers['Anomalies']==0)]
    if len(Outliers[(Outliers['Anomalies']==1)])>0:
        Y_anomalies=Outliers[(Outliers['Anomalies']==1)]
        for j in range(0,len(Y_anomalies)):
            line=[Y_anomalies.iloc[j].index[0],Y_anomalies.iloc[j].values[0],Y_anomalies.iloc[j].name]

            log_outliers.write(str(line)+"\n")
    Y_not_anomalies=Y_not_anomalies.drop(columns='Anomalies')
    Y_not_anomalies.index.name='time'
    Y_not_anomalies.columns=['tasmax']
    Y_not_anomalies.to_xarray().to_netcdf( os.path.join( pathOutY ,type_data_output, "full_Tricastin_ssp585_{}.nc".format(m) ) )
log_outliers.close()

In [95]:
#Enlever Outliers pour X
pathOutX= os.path.join(pathInp,"CMIP6/X")
log_outliers=open(pathOutX+'/X_Outliers_log.txt','a')
for i in range(0,len(models)):
    m=models[i]
    X=lX[i]
    Outliers=is_outlier(X, m)
    X_not_anomalies=Outliers[(Outliers['Anomalies']==0)]
    if len(Outliers[(Outliers['Anomalies']==1)])>0:
        X_anomalies=Outliers[(Outliers['Anomalies']==1)]
        for j in range(0,len(X_anomalies)):
            line=[X_anomalies.iloc[j].index[0],X_anomalies.iloc[j].values[0],X_anomalies.iloc[j].name]

            log_outliers.write(str(line)+"\n")
    X_not_anomalies=X_not_anomalies.drop(columns='Anomalies')
    X_not_anomalies.index.name='time'
    X_not_anomalies.columns=['tas']
    X_not_anomalies.to_xarray().to_netcdf( os.path.join( pathOutX ,type_data_output, "full_Europe_tas_YearMean_ssp585_{}.nc".format(m) ) )
log_outliers.close()

In [96]:
#Bon format
Xo.index=pd.to_datetime(Xo.index.astype(str), yearfirst=True)
Xo.index.name='time'
Xo.columns=['tas']
Xo.to_xarray().to_netcdf( os.path.join( pathInp ,"Observations",type_data_output,"Xo.nc") )

In [97]:
Yo.index=pd.to_datetime(Yo.index.astype(str), yearfirst=True)
Yo.index.name='time'
Yo.columns=['TX']
Yo.to_xarray().to_netcdf( os.path.join( pathInp ,"Observations",type_data_output,"Yo.nc") )